Example copy:

1. Torchaudio load from S3:
    - client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    - response = client.get_object(Bucket=S3_BUCKET, Key=S3_KEY)
    - waveform, sample_rate = torchaudio.load(response['Body'])
    - plot_specgram(waveform, sample_rate, title="From S3")

Pre installatie
- sudo apt install libsndfile1-dev
- sudo apt install awscli

- aws configure sso (Met informatie geleverd door Nick)

- conda create --name humainr python=3.9
- pip install boto3
- pip install transformers"[speech, sentencepiece]"
- pip install datasets


### Extra voor later

1. print(uuid.uuid4())


In [4]:
print(
    """\
Content-Type: text/html

<!DOCTYPE html>
<html lang="en">
<body>
<h1>Hello, World!</h1>
</body>
</html>"""
)

Content-Type: text/html

<!DOCTYPE html>
<html lang="en">
<body>
<h1>Hello, World!</h1>
</body>
</html>


Verschillende imports voor het programma

In [2]:
### General
import os
import torch
import uuid
import shutil
from io import BytesIO
import json

### s3 and DynamoDB
import boto3

### Transcibing trough ML
import torchaudio
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset, Audio


/home/arnold/anaconda3/envs/humainr/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuratie

In [3]:

boto3.setup_default_session(profile_name='AWSIntroTraining-408122842185')
s3 = boto3.resource('s3')
bucketname = 'humainr-aws-intro-bucket'
bucket = s3.Bucket(bucketname)

db = boto3.resource('dynamodb')
tablename = 'AWSIntroTable'
table = db.Table(tablename)

sync_path = 'data/new/'
arch_path = 'data/archive/'
test_file = 'common_voice_en_38497561.mp3'

for my_bucket_object in bucket.objects.all():
    print(my_bucket_object.key)


common_voice_en_38497561 (copy).mp3
common_voice_en_38497561.mp3
test.json


Verwerk de klaarstaande audiobestanden naar de AWS bucket

1 - Upload file naar S3
2 - Verplaats bestand van de sync_dir naar de arch_dir

In [6]:

print("Inhoud van de locale disk:")
for entry in os.listdir(sync_path):
    if entry.endswith('.mp3'):
        #bucket.upload_file(sync_path + entry, entry)
        #shutil.move(sync_path + entry, arch_path + entry)
        pass

print("Inhoud van de s3 bucket:")
for my_bucket_object in bucket.objects.all():
    print(my_bucket_object.key)
    ### Volgende regel is voor debug om de s3 bucket geheel leeg te gooien.
    #my_bucket_object.delete()



Inhoud van de locale disk:
Inhoud van de s3 bucket:
common_voice_en_38497561 (copy).mp3
common_voice_en_38497561.mp3
eb1cc731-9c78-4b55-8657-9ffeebbd1a11
test.json


Sectie voor de transcriptie:

1 - Zet bytestream om naar bruikbare data
2 - Pas de samplingrate aan
3 - Extraheer feautures
4 - Transcribeer de data

In [13]:
model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

### Laad de file/files in de dataset die transcribed moet worden
#ds = load_dataset("audiofolder", data_files=[sync_path + test_file], split="train")
obj = s3.Object(bucket_name=bucketname, key=test_file)
response = obj.get()
audio_bytes = BytesIO(response['Body'].read())

### Zet de bytestream van de s3 opslag om naar een numberarray
waveform, sample_rate = torchaudio.load(audio_bytes, format="mp3")

### Zet de sampling rate om naar 16000 voor het getrainde model
waveform = torchaudio.functional.resample(waveform, orig_freq=sample_rate, new_freq=16000)

### Genereer features uit de data
inputs = processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt")

### [TODO] Uitzoeken wat deze functie precies doet.
generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])

### [TODO] Omzetten prediction naar transcriptie.
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
transcription


Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.encoder.embed_positions.weights', 'model.decoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['they hastily drive to the plain and flee from the dangers of the mob']

### Opslaan transcriptie in DynamoDB


In [20]:
print(table.creation_date_time)
attrs = table.attribute_definitions
print(attrs)

table.put_item(
   Item={
        'id': '1234-56789',
        'file_name': 'test_file.bak',
        'framerate': '32000',
        'transcript': transcription[0],
    }
)

table.put_item(
   Item={
        'id': '5678-901234',
        'file_name': 'test_file.bak',
        'framerate': '32000',
        'transcript': transcription[0],
    }
)

response = table.get_item(
    Key={
        'id': '1234-56789',
    }
)
print(response['Item'])
print(response['Item']['transcript'])

response = table.scan()
print(response['Items'])



#response = table.delete_item(
#    Key={
#        'id': '1234-56789'
#    }
#)

response = table.get_item(
    Key={
        'id': '1234-56789',
    }
)
try:
    item = response['item']
except:
    print('no item found: 1234-56789')


2023-12-22 12:22:48.688000+01:00
[{'AttributeName': 'id', 'AttributeType': 'S'}]
{'framerate': '32000', 'id': '1234-56789', 'file_name': 'test_file.bak', 'transcript': 'they hastily drive to the plain and flee from the dangers of the mob'}
they hastily drive to the plain and flee from the dangers of the mob
[{'framerate': '32000', 'id': '5678-901234', 'file_name': 'test_file.bak', 'transcript': 'they hastily drive to the plain and flee from the dangers of the mob'}, {'framerate': '32000', 'id': '1234-56789', 'file_name': 'test_file.bak', 'transcript': 'they hastily drive to the plain and flee from the dangers of the mob'}]
no item found: 1234-56789


Verwerk de al verwerkte data naar een leesbare html pagina

In [17]:
# write-html.py

f = open('helloworld.html','w')

message = """
<html>
<head>MiniPIA</head>
<body>
    <p>Hello World!</p>
    """ + transcription[0] + """
</body>
</html>"""

f.write(message)
f.close()

#### Host de pagina's

In [11]:
import http.server
import socketserver

class MyHttpRequestHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.path = 'helloworld.html'
        return http.server.SimpleHTTPRequestHandler.do_GET(self)

# Create an object of the above class
handler_object = MyHttpRequestHandler

PORT = 8000
my_server = socketserver.TCPServer(("", PORT), handler_object)

# Star the server
my_server.serve_forever()

KeyboardInterrupt: 

Start html server op om verwerkte data te publiseren

[TODO] Pagina maken om data te uploaden en te gaan verwerken. (als er tijd over is)